# 控制释放试验 

In [46]:
import numpy as np
from osgeo import gdal, osr

import tempfile
import os
import sys

sys.path.append("C:\\Users\\RS\\VSCode\\matchedfiltermethod\\src")
from algorithms import matched_filter_variants as mfs
from algorithms import matched_filter_all as mfa
from utils import satellites_data as sd
from utils import generate_radiance_lut_and_uas as glut

import math


# 基于IME算法进行排放量的估算
def emission_estimate(plume_array, pixel_resolution, windspeed_10m, slope, intercept, enhancement_unit='ppmm'):
    # calculate the area and the length of the plume
    nan_count = np.count_nonzero(~np.isnan(plume_array))
    pixel_area = math.pow(pixel_resolution, 2)
    plume_area = nan_count * pixel_area
    plume_length = math.sqrt(plume_area)
    # get the values of the plume
    plume_values = [value for value in plume_array.flatten() if value != -9999]
    if enhancement_unit == 'ppmm':
        # convert the unit from  ppm*m to kg/ppm*m, then calculate the integrated mass enhancement
        integrated_mass_enhancement = sum(plume_values) * 0.716 * 0.000001 * pixel_area
    elif enhancement_unit == 'ppm':
        # convert the unit from  ppm*m to kg/ppm by setting 8km as the scale of troposphere,
        # then calculate the integrated mass enhancement
        integrated_mass_enhancement = sum(plume_values) * 0.716 * 0.000001 * pixel_area * 8000
    else:
        print("The unit of the enhancement is not supported, please enter 'ppmm' or 'ppm'.")
    # calculate the effective windspeed with the formula
    effective_windspeed = slope * windspeed_10m + intercept
    # calculate the emission rate of the plume in the unit of kg/h
    emission_rate = (effective_windspeed * 3600 * integrated_mass_enhancement) / plume_length
    return emission_rate


# PRISMA 数据


In [45]:
def PRISMA_test():
    filepath = "I:\stanford_campaign\PRISMA\PRS_L1_STD_OFFL_20221027182300_20221027182304_0001.he5"
    # filepath = "I:\stanford_campaign\PRISMA\PRS_L1_STD_OFFL_20221130180952_20221130180956_0001.he5"
    _,radiance_cube = sd.PRISMA_data.get_prisma_bands_array(filepath,2150,2500)
    sza,altitude = sd.PRISMA_data.get_prisma_SZA_altitude(filepath)
    if altitude > 5:
        altitude = 5
    _,uas = glut.generate_satellite_uas_for_specific_range_from_lut("PRISMA",0,30000,2150,2500,sza,altitude)


    # 原始匹配滤波算法结果测试
    # methane_enhancement = mfa.matched_filter(radiance_cube,uas, True, True)
    methane_enhancement = mfs.columnwise_matched_filter(radiance_cube,uas,True,True)
    # 多层匹配滤波算法结果测试
    # methane_enhancement = mfa.ml_matched_filter(radiance_cube,uas,True,True)
    # methane_enhancement = mfs.ml_matched_filter_new(radiance_cube,uas, True)

    # output = "I:\stanford_campaign\PRISMA\mf_PRS_L1_STD_OFFL_20221015181614_20221015181618_0001.tif"
    # output = "I:\stanford_campaign\PRISMA\mlmf_PRS_L1_STD_OFFL_20221015181614_20221015181618_0001.tif"
    
    output = "I:\stanford_campaign\PRISMA\stanford_campaign\PRISMA\PRS_L1_STD_OFFL_20221027182300_20221027182304_0001_cwmf.tif"
    # output = "I:\stanford_campaign\PRISMA\PRS_L1_STD_OFFL_20221130180952_20221130180956_0001_mf.tif"
    sd.PRISMA_data.location_calibration(methane_enhancement,filepath,output)
    return methane_enhancement

result = PRISMA_test()
print(result.max())

处理文件时出错: 'NoneType' object has no attribute 'SetGeoTransform'
117168.96582826281


In [ ]:
# 量化排放
plume_path = r"C:\Users\RS\Desktop\plumes\gf5_mf.tif"
plume_data = sd.general_functions.read_tiff_in_numpy(plume_path)
plume_data[plume_data == -3.4028231e+38] = np.nan
print(np.nanmax(plume_data))
fluxrate = emission_estimate(plume_data[plume_data >0],30,2.1274,1,0,"ppmm")
print(fluxrate/1000/8)

# AHSI 数据

In [35]:
def AHSI_test():
    filepath = "I:\stanford_campaign\Stanford_Campaign_GF5-02-AHSI\GF5B_AHSI_W112.1_N32.8_20221115_006332_L10000239663_VNSW_Rad.dat"
    _,radiance_cube = sd.AHSI_data.get_AHSI_radiances_from_dat(filepath,2150,2500)
    sza,altitude = sd.AHSI_data.get_sza_altitude(filepath)
    if altitude > 5:
        altitude = 5
    _,uas = glut.generate_satellite_uas_for_specific_range_from_lut("AHSI",0,50000,2150,2500,sza,altitude)

    # 原始匹配滤波算法结果测试
    # methane_enhancement = mfa.matched_filter(radiance_cube,uas, True, True)
    # 多层匹配滤波算法结果测试
    # methane_enhancement = mfs.ml_matched_filter_new(radiance_cube,uas, True)
    methane_enhancement = mfa.ml_matched_filter(radiance_cube,uas, True,True)
    
    # np.save("methane_enhancement.npy",methane_enhancement)
    
    # 假设 result_array 是处理后的结果数组
    result_array = methane_enhancement

    # 打开原始影像文件
    original_dataset = gdal.Open(filepath)

    rpc_info = original_dataset.GetMetadata("RPC")

    # 创建一个临时文件来存储结果数组
    temp_file = tempfile.NamedTemporaryFile(suffix=".tif").name
    driver = gdal.GetDriverByName("GTiff")
    output_dataset = driver.Create(temp_file, result_array.shape[1], result_array.shape[0], 1, gdal.GDT_Float32)

    # 将结果数组写入临时影像
    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(result_array)
    output_band.FlushCache()

    # 设置临时影像的地理信息
    output_dataset.SetProjection(original_dataset.GetProjection())
    output_dataset.SetGeoTransform(original_dataset.GetGeoTransform())

    # 将RPC信息写入临时文件的元数据
    output_dataset.SetMetadata(rpc_info, "RPC")

    # 关闭并保存临时影像文件
    output_dataset = None

    # 使用 WarpOptions 并从临时文件路径进行校正
    corrected_file = r"I:\stanford_campaign\Stanford_Campaign_GF5-02-AHSI\GF5B_AHSI_W112.1_N32.8_20221115_006332_L10000239663_VNSW_Rad_mlmf.tif"
    warp_options = gdal.WarpOptions(rpc=True)
    gdal.Warp(corrected_file, temp_file, options=warp_options)

    return methane_enhancement

# 获取反演结果
methane_enhancement = AHSI_test()



SZA: 52.831959
No significant plume detected.
Iteration 0: Converged with std change 14.38914742624172.


In [54]:

# 量化排放速率

plume_path = r"C:\Users\RS\Desktop\plumes\gf5_mf.tif"
plume_data = sd.general_functions.read_tiff_in_numpy(plume_path)
plume_data[plume_data == -3.4028231e+38] = np.nan
print(np.nanmax(plume_data))
fluxrate = emission_estimate(plume_data[plume_data >0],30,0.8656,1,0,"ppmm")
print(fluxrate/1000/8)


16191.056
0.4299618684004504


# EnMAP数据

In [48]:
# ! 需要修改
def EnMAP_test():
    # filepath = r"I:\stanford_campaign\EnMAP\dims_op_oc_oc-en_701862725_1\ENMAP.HSI.L1B\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T040610Z\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T040610Z-SPECTRAL_IMAGE_SWIR.TIF"
    # filepath = r"I:\stanford_campaign\EnMAP\dims_op_oc_oc-en_701862725_1\ENMAP.HSI.L1B\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T042058Z\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T042058Z-SPECTRAL_IMAGE_SWIR.TIF"
    filepath = r"I:\stanford_campaign\EnMAP\dims_op_oc_oc-en_701862725_1\ENMAP.HSI.L1B\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T042058Z\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T042058Z-SPECTRAL_IMAGE_SWIR.TIF"
    _,radiance_cube = sd.EnMAP_data.get_enmap_bands_array(filepath,2150,2500)
    sza = sd.EnMAP_data.get_center_sun_elevation(filepath.replace("SPECTRAL_IMAGE_SWIR.TIF", "METADATA.XML"))
    altitude = 0
    if altitude > 5:
        altitude = 5
    _,uas = glut.generate_satellite_uas_for_specific_range_from_lut("EnMAP",0,50000,2150,2500,sza,altitude)

    # 原始匹配滤波算法结果测试
    # methane_enhancement = mfa.matched_filter(radiance_cube,uas, True, True)
    methane_enhancement = mfs.columnwise_matched_filter(radiance_cube,uas, True, True)
    # 多层匹配滤波算法结果测试
    # methane_enhancement_mlmf = mfs.ml_matched_filter_new(radiance_cube,uas, True)
    
    output_folder = os.path.dirname(filepath)
    filename = os.path.basename(filepath)
    vnir =  r"I:\stanford_campaign\EnMAP\dims_op_oc_oc-en_701862725_1\ENMAP.HSI.L1B\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T042058Z\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T042058Z-SPECTRAL_IMAGE_VNIR.TIF"
   
    sd.EnMAP_data.export_enmap_array_to_tiff(methane_enhancement,filepath,output_folder,"1cwmf_" + filename)
    
    return methane_enhancement

result = EnMAP_test()

中心点太阳高度角: 37.826602 度
File saved successfully at I:\stanford_campaign\EnMAP\dims_op_oc_oc-en_701862725_1\ENMAP.HSI.L1B\ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T042058Z\1cwmf_ENMAP01-____L1B-DT0000005368_20221116T184050Z_005_V010501_20241017T042058Z-SPECTRAL_IMAGE_SWIR.TIF


In [52]:
# 量化排放
plume_path = r"C:\Users\RS\Desktop\plumes\enmap_mf.tif"
plume_data = sd.general_functions.read_tiff_in_numpy(plume_path)
plume_data[plume_data == -3.4028231e+38] = np.nan
print(np.nanmax(plume_data))
fluxrate = emission_estimate(plume_data[plume_data >0],30,1,1,0,"ppmm")
print(fluxrate/8000)


6628.8105
0.17409089445938705


# ZY1 数据

In [2]:
def ZY1_test():
    filepath = "I:\stanford_campaign\Stanford_Campaign_ZY1-02E-AHSI\ZY1F_AHSI_W111.72_N33.06_20221026_004370_L1A0000265656_VNSW_Rad.dat"
    # "I:\stanford_campaign\Stanford_Campaign_ZY1-02E-AHSI\ZY1F_AHSI_W111.74_N33.06_20221023_004327_L1A0000261518_VNSW_Rad.dat"
    # "I:\stanford_campaign\Stanford_Campaign_ZY1-02E-AHSI\ZY1F_AHSI_W111.85_N32.62_20221020_004284_L1A0000258234_VNSW_Rad.dat"
    _,radiance_cube = sd.ZY1_data.get_AHSI_radiances_from_dat(filepath,2150,2500)
    sza,altitude = sd.ZY1_data.get_sza_altitude(filepath)
    if altitude > 5:
        altitude = 5
    _,uas = glut.generate_satellite_uas_for_specific_range_from_lut("ZY1",0,50000,2150,2500,sza,altitude)

    # 原始匹配滤波算法结果测试
    methane_enhancement = mfa.matched_filter(radiance_cube,uas, True, True)
    # 多层匹配滤波算法结果测试
    # methane_enhancement_mlmf = mfs.ml_matched_filter_new(radiance_cube,uas, True)
    
    # np.save("methane_enhancement.npy",methane_enhancement)
    # 假设 result_array 是处理后的结果数组
    result_array = methane_enhancement

    # 打开原始影像文件
    original_dataset = gdal.Open(filepath)

    rpc_info = original_dataset.GetMetadata("RPC")

    # 创建一个临时文件来存储结果数组
    temp_file = tempfile.NamedTemporaryFile(suffix=".tif").name
    driver = gdal.GetDriverByName("GTiff")
    output_dataset = driver.Create(temp_file, result_array.shape[1], result_array.shape[0], 1, gdal.GDT_Float32)

    # 将结果数组写入临时影像
    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(result_array)
    output_band.FlushCache()

    # 设置临时影像的地理信息
    output_dataset.SetProjection(original_dataset.GetProjection())
    output_dataset.SetGeoTransform(original_dataset.GetGeoTransform())

    # 将RPC信息写入临时文件的元数据
    output_dataset.SetMetadata(rpc_info, "RPC")

    # 关闭并保存临时影像文件
    output_dataset = None

    # 使用 WarpOptions 并从临时文件路径进行校正
    corrected_file = r"I:\stanford_campaign\Stanford_Campaign_ZY1-02E-AHSI\ZY1F_AHSI_W111.72_N33.06_20221026_004370_L1A0000265656_VNSW_Rad_mf.tif"
    warp_options = gdal.WarpOptions(rpc=True)
    gdal.Warp(corrected_file, temp_file, options=warp_options)
    return methane_enhancement

methane_enhancement = ZY1_test()



SZA: 46.971901


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002D31DBE71E0> >

In [40]:
# 量化排放
plume_path = r"C:\Users\RS\Desktop\plumes\zy1_mf.tif"
plume_data = sd.general_functions.read_tiff_in_numpy(plume_path)
print(np.nanmax(plume_data))
fluxrate = emission_estimate(plume_data[plume_data >0],30,2.3443,1,0,"ppmm")
print(fluxrate/8000)

6475.968
0.45889248736911
